In [11]:
import pandas as pd
import json

In [2]:
df = pd.read_csv("trump_data.csv")

In [22]:
# keys in each json row
print(json.loads(df.iloc[0]['transcript'])[0].keys())

dict_keys(['time', 'speaker', 'remark'])


In [28]:
# Find unique types of events (debate, speech, interview, remarks, etc)
titles = set()
for idx, row in df.iterrows():

	if type(row['event_name']) is not str:
		print(row['event_name'])
	else:
		titles.add(row['event_name'].split(':')[0])
print(titles)

nan
nan
{'Debate', 'Convention', 'Remarks', 'Press Gaggle', 'Interview - PENDING', 'Press Briefing', 'Speech', 'Court Transcript', 'Press Conference', 'Interview', 'Donald Trump Vlog', 'Op-Ed'}


In [29]:
speakers = set()
for block in json.loads(df.iloc[0]['transcript']):
	speakers.add(block.get('speaker'))

print(speakers)

{'Pete Hegseth', 'Dan Caine', 'Tom Homan', 'Donald Trump', 'Question', 'Unidentified'}


In [31]:
def combine_blocks(transcript_list):
	if not transcript_list:
		return None

	merged = []
	cur = None

	for block in transcript_list:
		if cur is None:
			cur = block
			continue
		if block['speaker'] == cur['speaker']:
			cur['remark'] += " " + block['remark']
		else:
			merged.append(cur)
			cur = block
	if cur:
		merged.append(cur)
	return merged


In [35]:
print(df.iloc[3]['transcript'])

[{"time": "00:00:00", "speaker": "Question", "remark": "Mr. President, do you have a response to the attack in Syria?"}, {"time": "00:00:06", "speaker": "Donald Trump", "remark": "So we mourn the loss of three great patriots in Syria. You know how it happened. It was an ambush. It's terrible. We also have three wounded. They seem to be doing pretty well. But we mourn the loss. These are great -- three great people. And it's just a terrible thing. Syria, by the way, was fighting along with us. The President -- the new President of Syria is, as they told me, and I'm not surprised, he's devastated by what happened."}, {"time": "00:00:36", "speaker": "Donald Trump", "remark": "This was an ISIS attack on us and Syria. And again, we mourn the loss, and we pray for them and their parents and their loved ones. Very, very sad."}, {"time": "00:00:50", "speaker": "Question", "remark": "Mr. President, will the U.S. retaliate against ISIS?"}, {"time": "00:00:53", "speaker": "Donald Trump", "remark"

In [37]:
print(combine_blocks(json.loads(df.iloc[3]['transcript'])))

[{'time': '00:00:00', 'speaker': 'Question', 'remark': 'Mr. President, do you have a response to the attack in Syria?'}, {'time': '00:00:06', 'speaker': 'Donald Trump', 'remark': "So we mourn the loss of three great patriots in Syria. You know how it happened. It was an ambush. It's terrible. We also have three wounded. They seem to be doing pretty well. But we mourn the loss. These are great -- three great people. And it's just a terrible thing. Syria, by the way, was fighting along with us. The President -- the new President of Syria is, as they told me, and I'm not surprised, he's devastated by what happened. This was an ISIS attack on us and Syria. And again, we mourn the loss, and we pray for them and their parents and their loved ones. Very, very sad."}, {'time': '00:00:50', 'speaker': 'Question', 'remark': 'Mr. President, will the U.S. retaliate against ISIS?'}, {'time': '00:00:53', 'speaker': 'Donald Trump', 'remark': 'Yeah, we will.'}, {'time': '00:00:56', 'speaker': 'Question

In [38]:
df['transcript'] = df['transcript'].apply(lambda x: json.loads(x) if isinstance(x, str) else x)
df['transcript'] = df['transcript'].apply(combine_blocks)

print(f"Row 0 (Before): {len(json.loads(pd.read_csv('trump_data.csv').iloc[0]['transcript']))} blocks")
print(f"Row 0 (After):  {len(df.iloc[0]['transcript'])} blocks")

df['transcript'] = df['transcript'].apply(json.dumps)
df.to_csv('trump_data_clean.csv', index=False)

Row 0 (Before): 176 blocks
Row 0 (After):  94 blocks


In [39]:
fd = pd.read_csv('trump_data_clean.csv')
fd.columns

Index(['unique_id', 'date', 'event_name', 'url', 'transcript'], dtype='object')